# City Bid Tracker - Claremont

Automated scraper for public procurement opportunities from Claremont's official website.

## Purpose
Helps contractors and vendors discover bidding opportunities by extracting:
- Bid titles and descriptions
- Closing dates and status
- Direct links to full documentation

## Setup & Usage
1. Run the dependency installation cell
2. Execute the crawler class definition
3. Run the final execution cell
4. CSV file will be automatically downloaded

## Output
Creates `claremont_bids.csv` with all current bid opportunities.

## Technical Notes
This crawler handles a unique website structure that differs from standard bidding platforms, using custom CSS selectors for list containers instead of traditional table parsing.

In [ ]:
!pip install selenium webdriver_manager pandas

# Install Chrome and ChromeDriver
!apt-get update
!apt install chromium-chromedriver

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 61.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.9/492.9 kB 26.0 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,317 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from datetime import datetime
import csv
import os
import time
from google.colab import files
import random

class ClaremontBidsCrawler:
    def __init__(self):
        self.base_url = "https://www.claremontca.gov/Business-Development/City-Vending/Bids-and-Tenders-Current"
        self.output_file = "claremont_bids.csv"
        self.fieldnames = [
            "Category",
            "Bid Title",
            "Description",
            "Closing Date",
            "Status",
            "Details URL",
            "Last Updated"
        ]
        self.max_retries = 3
        self.setup_driver()

    def setup_driver(self):
        """Setup Chrome driver with enhanced options"""
        chrome_options = Options()
        chrome_options.add_argument('--headless')
        chrome_options.add_argument('--no-sandbox')
        chrome_options.add_argument('--disable-dev-shm-usage')
        chrome_options.add_argument('--disable-gpu')
        chrome_options.add_argument('--window-size=1920,1080')

        # Add realistic browser headers
        chrome_options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36')
        chrome_options.add_argument('--accept-language=en-US,en;q=0.9')
        chrome_options.add_argument('--disable-blink-features=AutomationControlled')

        try:
            print("Attempting to use system chromedriver...")
            self.driver = webdriver.Chrome(options=chrome_options)
        except Exception as e:
            print(f"System chromedriver failed: {str(e)}")
            print("Attempting to use ChromeDriverManager...")
            service = Service(ChromeDriverManager().install())
            self.driver = webdriver.Chrome(service=service, options=chrome_options)

        self.driver.set_page_load_timeout(30)
        self.wait = WebDriverWait(self.driver, 15)
        print("Chrome driver initialized successfully")

    def random_delay(self):
        """Add random delay between actions"""
        time.sleep(random.uniform(2, 5))

    def parse_bid_item(self, item, category=""):
        """Parse individual bid listing"""
        try:
            print("\nParsing new bid item...")
            bid_data = {
                "Category": category,
                "Bid Title": "",
                "Description": "",
                "Closing Date": "",
                "Status": "",
                "Details URL": "",
                "Last Updated": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            }

            try:
                # Get title, URL and description
                title_elem = item.find_element(By.CSS_SELECTOR, "h2.list-item-title")
                bid_data["Bid Title"] = title_elem.text.strip()

                link_elem = item.find_element(By.CSS_SELECTOR, "article a")
                bid_data["Details URL"] = link_elem.get_attribute("href")

                desc_elem = item.find_element(By.CSS_SELECTOR, "article p:not([class])")
                if desc_elem:
                    bid_data["Description"] = desc_elem.text.strip()

                # Get status and closing date
                status_elem = item.find_element(By.CSS_SELECTOR, "p.status-list")
                if status_elem:
                    status_text = status_elem.text.strip()
                    if "Status:" in status_text:
                        bid_data["Status"] = status_text.split("Status:")[1].strip()

                closing_elem = item.find_element(By.CSS_SELECTOR, "p.applications-closing")
                if closing_elem:
                    closing_text = closing_elem.text.strip()
                    if "Closed" in closing_text:
                        bid_data["Closing Date"] = closing_text.replace("Closed", "").strip()

            except Exception as e:
                print(f"Error parsing bid elements: {str(e)}")
                return None

            print(f"Parsed bid: {bid_data['Bid Title']}")
            return bid_data if bid_data["Bid Title"] else None

        except Exception as e:
            print(f"Error parsing bid item: {str(e)}")
            return None

    def setup_csv(self):
        """Create or verify CSV file with headers"""
        try:
            if not os.path.exists(self.output_file):
                with open(self.output_file, 'w', newline='', encoding='utf-8') as f:
                    writer = csv.DictWriter(f, fieldnames=self.fieldnames)
                    writer.writeheader()
                print(f"Created new CSV file: {self.output_file}")
            else:
                print(f"CSV file already exists: {self.output_file}")
        except Exception as e:
            print(f"Error setting up CSV: {str(e)}")

    def get_page_with_retry(self):
        """Attempt to load the page with retries"""
        for attempt in range(self.max_retries):
            try:
                print(f"\nAttempt {attempt + 1} to load page...")
                self.driver.get(self.base_url)
                self.random_delay()

                # Check if page loaded successfully
                try:
                    self.wait.until(
                        EC.presence_of_element_located((By.CLASS_NAME, "list-container"))
                    )
                except Exception as e:
                    print(f"Error waiting for page load: {str(e)}")
                    continue

                print("Page loaded successfully")
                return True

            except Exception as e:
                print(f"Error loading page: {str(e)}")
                if attempt < self.max_retries - 1:
                    wait_time = (attempt + 1) * 5
                    print(f"Waiting {wait_time} seconds before retry...")
                    time.sleep(wait_time)
                continue
        return False

    def get_bid_listings(self):
        """Fetch and parse all bid listings"""
        try:
            if not self.get_page_with_retry():
                print("Failed to load page after all retries")
                return []

            print("Looking for bid listings...")
            bids = []

            # Find the list container and all bid items
            list_container = self.wait.until(
                EC.presence_of_element_located((By.CLASS_NAME, "list-container"))
            )

            bid_items = list_container.find_elements(By.CLASS_NAME, "list-item-container")
            print(f"Found {len(bid_items)} bid items")

            for item in bid_items:
                bid_data = self.parse_bid_item(item)
                if bid_data:
                    bids.append(bid_data)
                self.random_delay()

            print(f"Successfully parsed {len(bids)} bids")
            return bids

        except Exception as e:
            print(f"Error fetching bid listings: {str(e)}")
            return []

    def save_bids(self, bids):
        """Save bid data to CSV"""
        try:
            if not bids:
                print("No bids to save")
                return

            existing_bids = set()
            if os.path.exists(self.output_file):
                with open(self.output_file, 'r', encoding='utf-8') as f:
                    reader = csv.DictReader(f)
                    for row in reader:
                        existing_bids.add(f"{row['Bid Title']}-{row['Closing Date']}")

            new_bids = [
                bid for bid in bids
                if f"{bid['Bid Title']}-{bid['Closing Date']}" not in existing_bids
            ]

            if new_bids:
                mode = 'w' if not os.path.exists(self.output_file) else 'a'
                with open(self.output_file, mode, newline='', encoding='utf-8') as f:
                    writer = csv.DictWriter(f, fieldnames=self.fieldnames)
                    if mode == 'w':
                        writer.writeheader()
                    writer.writerows(new_bids)
                print(f"Added {len(new_bids)} new bids")
            else:
                print("No new bids to add")

            # Download the CSV file
            files.download(self.output_file)

        except Exception as e:
            print(f"Error saving bids: {str(e)}")

    def run(self):
        """Main execution method"""
        try:
            print(f"Starting Claremont bids crawler at {datetime.now()}")
            self.setup_csv()
            bids = self.get_bid_listings()
            self.save_bids(bids)
            print("Crawler execution completed")
        finally:
            if hasattr(self, 'driver'):
                self.driver.quit()

# Example usage:
# crawler = ClaremontBidsCrawler()
# crawler.run()

In [ ]:
crawler = ClaremontBidsCrawler()
crawler.run()

Attempting to use system chromedriver...
Chrome driver initialized successfully
Starting Claremont bids crawler at 2025-02-20 16:10:14.474414
Created new CSV file: claremont_bids.csv

Attempt 1 to load page...
Page loaded successfully
Looking for bid listings...
Found 2 bid items

Parsing new bid item...
Parsed bid: Sidewalk Rehabilitation Project


## Disclaimer
This tool accesses publicly available information only from official government websites. It respects robots.txt guidelines and implements responsible scraping practices with delays between requests.